In [173]:
# Import Dependencies
import pandas as pd
from datetime import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
import os
import numpy as np

# Google Office Parsing

In [2]:
url='https://about.google/locations/?region=true'
response = requests.get(url)

In [3]:
soup = bs(response.text, 'html.parser')

In [4]:
results = soup.find_all("div", class_= "office-info")
results

[<div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Dubai
                 </h2>
 <div class="office-address" itemprop="address">TECOM Zone, Dubai Internet City
 Dubai, United Arab Emirates</div>
 <div class="office-phone-number">
                     Phone:
                     <span class="phone-number" data-phone-number="+971 4 4509500" itemprop="telephone">
                       +971 4 4509500
                     </span>
 </div>
 <div class="directions">
 <a href="https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en" rel="noopener" target="_blank">
                     Directions
                   </a>
 </div>
 </div>,
 <div class="office-info">
 <h2 class="office-name" itemprop="name">
                   Haifa
                 </h2>
 <div class="office-address" itemprop="address">Building 30
 MATAM, Advanced Technology Center
 Haifa, 3190500
 Israel </div>
 <div class="office-phone-number">
                     Phone:
      

In [152]:
results = soup.find_all("div", class_= "office-info")
for result in results:
    office = result.find("h2", itemprop= "name").text
    address = result.find("div",itemprop= "address").text
    quards = result.find("a")['href']
    print("-------------------")
    print(office, address, quards)
    
    

-------------------

                  Dubai
                 TECOM Zone, Dubai Internet City
Dubai, United Arab Emirates https://www.google.com/maps/dir/Current+Location/25.0929,55.1591?hl=en
-------------------

                  Haifa
                 Building 30
MATAM, Advanced Technology Center
Haifa, 3190500
Israel  https://www.google.com/maps/dir/Current+Location/32.78897,34.958432?hl=en
-------------------

                  Istanbul
                 Eski Buyukdere Caddesi No: 209
34394
Istanbul, Turkey https://www.google.com/maps/dir/Current+Location/41.081697,29.00859?hl=en
-------------------

                  Johannesburg
                 35 Ballyclare Drive Building E
Johannesburg
2191, South Africa https://www.google.com/maps/dir/Current+Location/-26.0734457,28.032035?hl=en
-------------------

                  Tel Aviv
                 Yigal Alon 98
Tel Aviv, 6789141
Israel  https://www.google.com/maps/dir/Current+Location/32.070043,34.794087?hl=en
-------------------


In [196]:
location = []
addresses = []
quardss = []

for result in results:
    office = result.find("h2", class_= "office-name").text
    address = result.find("div", class_= "office-address").text
    quards = result.find("a")['href']
    location.append(office)
    addresses.append(address)
    quardss.append(quards)
    

In [197]:
# Create a dataframe from the parsed data above

df = pd.DataFrame({
    "Office": location,
    "Address": addresses,
    "Quards": quardss
})

# Removes all occurances of "\n"
df["Office"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
df["Address"].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

#Drops the:'https://www.google.com/maps/dir/Current+Location/ at the beggining of the string
# and the: ?hl=en at the end of the string
df["Quards2"]= df["Quards"].str.strip('https://www.google.com/maps/dir/Current+Location/ ?hl=en')

# Drops the old Column with the google hyperlinks
df = df.drop(columns=['Quards'])


# new data frame with split value columns 
new = df["Quards2"].str.split(",", n = 1, expand = True) 
  
# making separate Latitude column from new data frame 
df["Latitude"]= new[0] 
  
# making separate Longitude column from new data frame 
df["Longitude"]= new[1] 
  
# Dropping old Quards2 columns 
df.drop(columns =["Quards2"], inplace = True) 
  
# Drops duplicates and only keeps the Office's first occurance
df= df.drop_duplicates(subset=["Office"], keep='first')

In [198]:
df["Company"] = ("Google")

In [199]:
df.set_index("Company")

,Office,Address,Latitude,Longitude
Company,,,,
Google,Dubai,"TECOM Zone, Dubai Internet City Dubai, United ...",25.0929,55.1591
Google,Haifa,"Building 30 MATAM, Advanced Technology Center ...",32.78897,34.958432
Google,Istanbul,"Eski Buyukdere Caddesi No: 209 34394 Istanbul,...",41.081697,29.00859
Google,Johannesburg,35 Ballyclare Drive Building E Johannesburg 21...,-26.0734457,28.032035
Google,Tel Aviv,"Yigal Alon 98 Tel Aviv, 6789141 Israel",32.070043,34.794087
...,...,...,...,...
Google,San Francisco,"345 Spear Street San Francisco, CA 94105 Unite...",37.789972,-122.390013
Google,Seattle,"601 N. 34th Street Seattle, WA 98103 United St...",47.649316,-122.350629
Google,Sunnyvale,"803 11th Avenue Sunnyvale, CA 94089 United States",37.403694,-122.031583
